In [2]:
import pygame
import random
import math
import time
import queue
import os
import boto3
import pandas as pd
from io import StringIO

pygame 2.6.1 (SDL 2.28.4, Python 3.12.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# Create a session using your configured AWS credentials

s3 = boto3.client('s3')

# Upload a file to S3
bucket_name = "trackage.1"
file_name = "luggage_dataset (3-4)350(0.75).csv"  # Path to your local file
s3_object_name = "database.csv"  # Name in S3

s3.upload_file(file_name, bucket_name, s3_object_name)

print(f"Uploaded {file_name} to S3 bucket {bucket_name} as {s3_object_name}")


# Download the file from S3
obj = s3.get_object(Bucket=bucket_name, Key=s3_object_name)

# Read it as a Pandas DataFrame
df = pd.read_csv(obj["Body"], delimiter=",", on_bad_lines="skip")

print(df.head())  # Show first 5 rows

print(df.columns)

df['arrival_time'] = df['Number_of_Bags'] * 2  

# Save the updated DataFrame to a temporary local file
temp_file = '/tmp/modified_dataset.csv'  # Local path to save the modified file
df.to_csv(temp_file, index=False)

# Upload the updated file back to the same location on S3 (this will overwrite the file)
s3.upload_file(temp_file, bucket_name, s3_object_name)

print(f"File {s3_object_name} updated successfully in S3.")


NameError: name 'boto3' is not defined

In [3]:
conveyor_belts = [
    [(1200, 600), (100, 600)],
    [(100, 550), (800, 550)],
    [(200, 500), (800, 500)],
    [(200, 450), (700, 450)],
    [(300, 400), (700, 400)],
    [(300, 350), (600, 350)],
    [(100, 250), (900, 250)],
    [(100, 600), (100, 200)],
    [(200, 500), (200, 200)],
    [(300, 400), (300, 200)],
    [(600, 350), (600, 200)],
    [(700, 450), (700, 200)],
    [(800, 550), (800, 200)],
    [(900, 600), (900, 250)],
    [(580, 200), (600, 250)],
    [(620, 200), (600, 250)],
    [(320, 200), (300, 250)],
    [(280, 200), (300, 250)],
    [(680, 200), (700, 250)],
    [(720, 200), (700, 250)],
    [(180, 200), (200, 250)],
    [(220, 200), (200, 250)],
    [(780, 200), (800, 250)],
    [(820, 200), (800, 250)],
    [(80, 200), (100, 250)],
    [(120, 200), (100, 250)],
]

points = [
    (100, 250), (100, 550), (100, 600),
    (200, 250), (200, 450), (200, 500),
    (300, 250), (300, 350), (300, 400),
    (600, 250), (600, 350), (700, 250),
    (700, 400), (700, 450), (800, 250),
    (800, 500), (800, 550), (900, 250),
    (900, 600)
]

gates_entry_positions = [
    (600, 250), # Gate 2
    (300, 250), # Gate 3
    (700, 250), # Gate 4
    (200, 250), # Gate 5
    (800, 250), # Gate 6
    (100, 250), # Gate 7
]

pickup_gates_positions = [
    (600, 200), # Gates 2
    (580, 200),
    (620, 200),
    (300, 200), # Gates 3
    (320, 200),
    (280, 200),
    (700, 200), # Gates 4
    (680, 200),
    (720, 200),
    (200, 200), # Gates 5
    (180, 200),
    (220, 200),
    (800, 200), # Gates 6
    (780, 200), 
    (820, 200),
    (100, 200), # Gates 7
    (80, 200),
    (120, 200),
]

In [4]:
class Path:
    
    def __init__(self, name, points):
        self.name = name  
        self.points = points
    
paths = {
    cluster: Path(f"{cluster}", points) for cluster, points in {
        7: [(900, 600), (100, 600), (100, 550), (100, 250)],
        6: [(900, 600), (100, 600), (100, 550), (800, 550), (800, 500), (800, 250)], 
        5: [(900, 600), (100, 600), (100, 550), (800, 550), (800, 500), (200, 500), (200, 450), (200, 250)], 
        4: [(900, 600), (100, 600), (100, 550), (800, 550), (800, 500), (200, 500), (200, 450), (700, 450), 
            (700, 400), (700, 250)],
        3: [(900, 600), (100, 600), (100, 550), (800, 550), (800, 500), (200, 500), (200, 450), (700, 450), 
            (700, 400), (300, 400), (300, 350), (300, 250)],
        2: [(900, 600), (100, 600), (100, 550), (800, 550), (800, 500), (200, 500), (200, 450), (700, 450), 
            (700, 400), (300, 400), (300, 350), (600, 350), (600, 250)]
    }.items()
}

paths_2 = {
  cluster: Path(f"{cluster}", points) for cluster, points in {
        7: [(900, 250), (900, 600)],
        6: [(900, 250), (900, 600)], 
        5: [(900, 250), (900, 600)], 
        4: [(900, 250), (900, 600)],
        3: [(900, 250), (900, 600)],
        2: [(900, 250), (900, 600)]
    }.items()
}

def generate_random_color():
    return (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))

cluster_sizes = [2, 3, 4, 5, 6, 7, 8, 9, 10] 

cluster_colors = {size: generate_random_color() for size in cluster_sizes}

PPM = 1100/170 

pixel_speed = ((3*PPM)/60)*20

In [5]:
class Bag:  
        
    def __init__(self, bag_id, reservation_id, cluster_size):
        self.ready = False
        self.path = []
        self.bag_id = bag_id
        self.reservation = reservation_id
        self.cluster_size = cluster_size
        self.position = (1200, 600)
        self.assigned_to_gate = False
        self.gate = None
        self.speed = pixel_speed
        self.target = None 
        self.color = cluster_colors[cluster_size]
        self.at_gate = False

    def set_at_gate(self):
        self.at_gate = True

    def set_ready(self):
        self.ready = True   

    def set_path(self):
        if(self.cluster_size < 8):
            self.path = paths[self.cluster_size].points[:]  
            if self.path:
                self.target = self.path.pop(0) 
        else:
            abs_path = self.cluster_size%7 + 1
            self.path = paths[abs_path].points[:]  
            if self.path:
                self.target = self.path.pop(0) 


    def set_gate(self, pickup_gate):
        self.gate = pickup_gate

    def append_path(self, path):
        if isinstance(path, Path):  
            self.path.extend(path.points)  
        elif isinstance(path, list):  
            self.path.extend(path)  
        elif isinstance(path, tuple):
            self.path.append(path)
        
        if not self.target and self.path:
            self.target = self.path.pop(0) # Get it moving

    def move(self):
        if self.target:
            x, y = self.position
            tx, ty = self.target
    
            dx = tx - x
            dy = ty - y
            distance = (dx**2 + dy**2) ** 0.5
    
            if distance <= self.speed:
                self.position = self.target  
                if self.path:
                    self.target = self.path.pop(0)  
                else:
                    self.target = None  
            else:
                self.position = (x + (dx / distance) * self.speed, 
                                 y + (dy / distance) * self.speed)


In [6]:
class Gate:
    
    def __init__(self, name, gates, position):
        self.name = name 
        self.gates = gates
        self.position = position
        self.res_queue = queue.Queue() 

    def add_res(self, reservation):
        if not reservation in list(self.res_queue.queue):
            self.res_queue.put(reservation)

    def search(self, bag):
        for i in range(3):
            if(self.gates[i].reservation == bag.reservation):
                return i
        for i in range(3):
            if(self.gates[i].reservation == None):
                return i
        return -1
   

In [7]:
class PickupGate:
    
    def __init__(self, name, position):
        self.name = name 
        self.position = position
        self.reservation = None
        self.load = 0
        self.max_load = 10
        self.bags = []
        self.full = False 
    
    def set_reservation(self, reservation):
        if self.reservation is None:  
            self.reservation = reservation
            return True
        else:
            return False

    def add_luggage(self, bag):
        if bag.reservation == self.reservation:
            self.bags.append(bag)
            self.load += 1
            
    def is_full(self):
        if(self.load>0):
            if self.load == self.bags[0].cluster_size:
                return True
        return False

    def clear_luggage(self):
        removed_bags = self.bags if self.bags else []  
        self.reservation = None
        self.load = 0
        self.bags = [] 
        return removed_bags  


In [8]:
gate_entries = []
k = 0
for i in range(6):
    gates = []
    for j in range(3):
        gates.append(PickupGate(j+1, pickup_gates_positions[k]))
        k += 1
    gate_entries.append(Gate(i+2, gates, gates_entry_positions[i]))

num_reservations = 30 
bag_id = 1
reservations = []
conveyor_belt = []

special_reservation_id = random.randint(1, num_reservations)  

# Iterate through the dataset to get reservations and cluster sizes
for index, row in df.iterrows():
    reservation_id = int(row['reservation_id'])  # Adjust column name as per dataset
    cluster_size = int(row['cluster_size'])  # Adjust column name as per dataset  

    reservations.append((reservation_id, cluster_size))

    for _ in range(cluster_size):
        conveyor_belt.append(Bag(bag_id, reservation_id, cluster_size))
        bag_id += 1

random.shuffle(conveyor_belt)

In [9]:
pygame.init()

width, height = 1600, 800
screen = pygame.display.set_mode((width, height))
pygame.display.set_caption("SLHS Visualization")

WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)

In [10]:

scanned_bags = []
i=0

clock = pygame.time.Clock()
running = True
last_spawn_time = 0  
spawn_interval = 2000 
font = pygame.font.Font(None, 20)
bag_start_times = []  
start_time = pygame.time.get_ticks()  
clock = pygame.time.Clock()

while running:
    screen.fill(WHITE) 
    clock.tick(60)
    for belt in conveyor_belts:
        pygame.draw.line(screen, BLACK, belt[0], belt[1], 3)

    for point in points:
        pygame.draw.circle(screen, BLACK, point, 5)

    for gate in gates_entry_positions:
        pygame.draw.rect(screen, BLACK, (gate[0] - 5, gate[1] - 5, 10, 10))

    for pickup in pickup_gates_positions:
        pygame.draw.rect(screen, BLACK, (pickup[0] - 5, pickup[1] - 5, 10, 10))

    for gate_group in gate_entries: 
        for gate in gate_group.gates: 
            load_text = font.render(f"{gate.load}", True, BLACK)    
            screen.blit(load_text, (gate.position[0]-5, 177))
        
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
            
    more_items = len(conveyor_belt)-1

    current_time = pygame.time.get_ticks()
    
    if i < len(conveyor_belt):  
        read_bag = conveyor_belt[i]  
        scanned_bags.append(read_bag)
        read_bag.set_ready()
        
        cluster_size = read_bag.cluster_size
        
        if cluster_size>7:
            cluster_size = cluster_size%7+1
        
        read_bag.set_path()
    
        gate = gate_entries[cluster_size-2].search(read_bag)

        delay_time = 500 
        bag_start_times.append(start_time+(i*delay_time))
        
        if gate == -1:
            gate_entries[cluster_size-2].add_res(read_bag.reservation)
        else:
            gate_entries[cluster_size-2].gates[gate].set_reservation(read_bag.reservation)
            read_bag.append_path(gate_entries[cluster_size-2].gates[gate].position)
            read_bag.set_gate(gate_entries[cluster_size-2].gates[gate])
            read_bag.assigned_to_gate = True
        
        i+=1
        last_spawn_time = current_time

    still_moving = False 

    j=0
    for bag in scanned_bags:
        
        if bag.cluster_size<8:            
            if bag.assigned_to_gate and bag.position == gate_entries[bag.cluster_size-2].gates[bag.gate.name-1].position and bag.at_gate:
                pygame.draw.circle(screen, bag.color, bag.position, 8)
            elif bag.assigned_to_gate and bag.position == gate_entries[bag.cluster_size-2].gates[bag.gate.name-1].position:
                gate_entries[bag.cluster_size-2].gates[bag.gate.name-1].add_luggage(bag)
                bag.set_at_gate()
                pygame.draw.circle(screen, bag.color, bag.position, 8)
            elif not bag.assigned_to_gate and bag.position == gate_entries[bag.cluster_size-2].position:
                bag.append_path(paths_2[bag.cluster_size])
                bag.append_path(paths[bag.cluster_size])
                if bag.path: 
                    bag.move()
                    pygame.draw.circle(screen, bag.color, bag.position, 8)
            elif current_time >= bag_start_times[j]:
                bag.move()
                pygame.draw.circle(screen, bag.color, bag.position, 8)
                still_moving = True 

        else:
            if bag.assigned_to_gate and bag.position == gate_entries[bag.cluster_size%7-1].gates[bag.gate.name-1].position and bag.at_gate:
                pygame.draw.circle(screen, bag.color, bag.position, 8)
            elif bag.assigned_to_gate and bag.position == gate_entries[bag.cluster_size%7-1].gates[bag.gate.name-1].position:
                gate_entries[bag.cluster_size%7-1].gates[bag.gate.name-1].add_luggage(bag)
                bag.set_at_gate()
                pygame.draw.circle(screen, bag.color, bag.position, 8)
            elif not bag.assigned_to_gate and bag.position == gate_entries[bag.cluster_size%7-1].position:
                bag.append_path(paths_2[bag.cluster_size%7-1])
                bag.append_path(paths[bag.cluster_size%7-1])
                if bag.path: 
                    bag.move()
                    pygame.draw.circle(screen, bag.color, bag.position, 8)
            elif current_time >= bag_start_times[j]:
                bag.move()
                pygame.draw.circle(screen, bag.color, bag.position, 8)
                still_moving = True 

        
        for gate_group in gate_entries: 
            for gate in gate_group.gates: 
                if gate.is_full() and random.random() < 0.00005: 
                    removed_bags = gate.clear_luggage()  
                    scanned_bags = [bag for bag in scanned_bags if bag not in removed_bags]  
                    print(f" Gate {gate_group.name} was emptied, and {len(removed_bags)} bags disappeared!")  

        for gate_group in gate_entries: 
            for gate in gate_group.gates: 
                if gate.reservation == None: 
                    if not gate_group.res_queue.empty():
                        res = gate_group.res_queue.get() 
                        gate.set_reservation(res)

        for gate_group in gate_entries: 
            for gate in gate_group.gates: 
                if not bag.assigned_to_gate and bag.reservation == gate.reservation:
                    bag.append_path(gate.position)
                    bag.set_gate(gate)
                    bag.assigned_to_gate = True

        j+=1

    pygame.display.update()
    clock.tick(60) 

    
    if i >= len(conveyor_belt) and not still_moving:
        running = False  

pygame.quit()


2025-03-22 22:51:00.790 python[34259:2127381] +[IMKClient subclass]: chose IMKClient_Modern
2025-03-22 22:51:00.791 python[34259:2127381] +[IMKInputSession subclass]: chose IMKInputSession_Modern


 Gate 2 was emptied, and 2 bags disappeared!
 Gate 2 was emptied, and 2 bags disappeared!
 Gate 2 was emptied, and 2 bags disappeared!
 Gate 7 was emptied, and 7 bags disappeared!
 Gate 2 was emptied, and 2 bags disappeared!
 Gate 6 was emptied, and 6 bags disappeared!
 Gate 4 was emptied, and 10 bags disappeared!
 Gate 7 was emptied, and 7 bags disappeared!
 Gate 6 was emptied, and 6 bags disappeared!
 Gate 7 was emptied, and 7 bags disappeared!
 Gate 4 was emptied, and 4 bags disappeared!
 Gate 3 was emptied, and 3 bags disappeared!
 Gate 5 was emptied, and 5 bags disappeared!
 Gate 3 was emptied, and 3 bags disappeared!
 Gate 7 was emptied, and 7 bags disappeared!
 Gate 2 was emptied, and 2 bags disappeared!
 Gate 4 was emptied, and 4 bags disappeared!
 Gate 4 was emptied, and 4 bags disappeared!
 Gate 2 was emptied, and 2 bags disappeared!
 Gate 3 was emptied, and 3 bags disappeared!
 Gate 5 was emptied, and 5 bags disappeared!
 Gate 5 was emptied, and 5 bags disappeared!
 Gate 4 w